In [1]:
import pandas as pd
import requests
import urllib3
import time

# Disable SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# Define the endpoints
endpoints = ["http://127.0.0.1:8000/ask"]

# Load the Excel file
input_file = "data-test.xlsx"
output_file = "nova_pro_optimized_generated_data.xlsx"
dataframe = pd.read_excel(input_file, engine="openpyxl")

# Ensure the 'user_input' column exists
if 'user_input' not in dataframe.columns:
    raise ValueError("The Excel file must contain a column named 'user_input'.")

# Prepare response and timing columns
dataframe["cleaned_output"] = ""
dataframe["ai_response"] = ""
dataframe["ai_time"] = 0.0

In [2]:

# Function to get responses and timing
def get_responses_with_time(question):
    payload = {
        "question": question,
        "conversation_history": [{"role": "", "content": ""}],
        "regenerate": False
    }
    responses = {}
    times = {}
    for url in endpoints:
        start_time = time.time()
        try:
            response = requests.post(url, json=payload, verify=False)
            response.raise_for_status()
            responses[url] = response.json().get("answer", "No answer returned")
        except requests.exceptions.HTTPError as http_err:
            responses[url] = f"HTTP error: {http_err} - Response: {response.text}"
        except Exception as e:
            responses[url] = f"Error: {e}"
        finally:
            times[url] = round(time.time() - start_time, 3)  # seconds
    return responses, times

In [3]:
from crewai.tools import BaseTool
import json
from crewai_tools.aws.bedrock.knowledge_base.retriever_tool import BedrockKBRetrieverTool
import boto3
from langchain_aws import BedrockEmbeddings
from opensearchpy import RequestsHttpConnection
from requests_aws4auth import AWS4Auth
from langchain_community.vectorstores import OpenSearchVectorSearch
import os
from typing import Type
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import time
import concurrent.futures

load_dotenv()

# AWS credentials and region
AWS_REGION = "us-east-1"
OPENSEARCH_ENDPOINT = "https://fkakzty7t3bgtpwvdjp3.us-east-1.aoss.amazonaws.com"

session = boto3.Session(
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)
credentials = session.get_credentials()
awsauth = AWS4Auth(
    credentials.access_key,
    credentials.secret_key,
    AWS_REGION,
    "aoss",
    session_token=credentials.token
)

# Bedrock client and embeddings
bedrock_client = boto3.client(
    service_name='bedrock-runtime',
    region_name=AWS_REGION,
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY')
)

embeddings = BedrockEmbeddings(
    client=bedrock_client,
    region_name=AWS_REGION,
    model_id="amazon.titan-embed-text-v2:0"
)

# Knowledge base retriever
kb_tool = BedrockKBRetrieverTool(
    knowledge_base_id="01ANQMZB9P",
    number_of_results=2,
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_region=AWS_REGION
)

kb_tool2 = BedrockKBRetrieverTool(
    knowledge_base_id="01ANQMZB9P",
    retrieval_configuration = {
  "vectorSearchConfiguration": {
     "numberOfResults": 2, 
     "overrideSearchType": "HYBRID",  
     "filter": {  
             "equals": {"key": "x-amz-bedrock-kb-source-uri", "value": "https://cbre-jira.atlassian.net/wiki/spaces/CLOUD/pages/39605931/Cloud+Platform+Request+Forms"}, 
     
     }
  }
},
    aws_secret_access_key=os.getenv('AWS_SECRET_ACCESS_KEY'),
    aws_access_key_id=os.getenv('AWS_ACCESS_KEY_ID'),
    aws_region=AWS_REGION
)

class RetrieveContentToolInput(BaseModel):
    data: str = Field(..., description="The data to process and extract content from.")

class RetrieveContentTool(BaseTool):
    name: str = "RetrieveContentTool"
    # description: str = "Extracts and returns content from both OpenSearch and Bedrock Knowledge Base."
    # description: str = (
    #     "Extracts and returns content from both OpenSearch and Bedrock Knowledge Base. "
    #     "Tool Arguments: {'query':  'kwargs': {performanceConfig={"latency": "optimized"}}}"
    #     "\n\nExpected input: Pass your query as a string to the 'data' argument. "
    #     "Example: tool_com(data='your question here')\n"
    #     "Do NOT pass a dictionary or use a 'query' key. Only a string is accepted."
    # )

    description: str = (
        "Extracts and returns content from both OpenSearch and Bedrock Knowledge Base. "
        "Tool Arguments: {'query': <string>, 'kwargs': {'performanceConfig': {'latency': 'optimized'}}}"
        "\n\nExpected input: Pass your query as a string to the 'data' argument. "
        "Example: tool_com(data='your question here')\n"
        "Do NOT pass a dictionary or use a 'query' key. Only a string is accepted."
    )

    args_schema: Type[BaseModel] = RetrieveContentToolInput
    
    def _run(self, data: str):
        overall_start = time.time()
        with concurrent.futures.ThreadPoolExecutor() as executor:
            future_opensearch = executor.submit(self.extract_from_opensearch, data)
            future_kb = executor.submit(self.extract_from_kb, data)
            future_kb2 = executor.submit(self.extract_from_kb2, data)
            opensearch_results = future_opensearch.result()
            kb_content= future_kb.result()
            kb_content2 = future_kb2.result()
        print(f"Total time taken for all operations: {time.time() - overall_start:.2f} seconds")
        
        # Format OpenSearch results
        opensearch_str = "=== ServiceNow Historical Tickets ===\n"
        if opensearch_results == []:
            opensearch_str += "No relevant ServiceNow historical tickets found.\n"
            
        for item in opensearch_results:
            opensearch_str += f"Ticket: {item['Ticket']}\nDescription: {item['Description'][0]}\n---\n"

        # Format KB results
        kb_str = "=== Confluence & SharePoint Results ===\n"
        
        for item in kb_content:
            kb_str += (
                f"Content: {item['content']}\n"
                f"Description: {item['description']}\n"
                f"Link: {item['url']}\n---\n"
            )

        # Format KB2 results
        kb2_str = "=== ServiceNow Reference Links ===\n"
        for item in kb_content2:
            kb2_str += (
                f"Content: {item['content']}\n"
                f"Link: {item['url']}\n---\n"
            )
        if not opensearch_results:
            result_str = f"{kb_str}\n{kb2_str}"
        else:
            result_str = f"{opensearch_str}\n{kb_str}\n{kb2_str}"
        return result_str
        # return {
        #     "Results from opensearch which have all the relevant information of service now historical tickets": opensearch_results,
        #     "Results from confluence which have all the relevant information of confluence and sharepoint data": kb_content,
        #     # "Image data Results from confluence which have all the relevant information of confluence and sharepoint data": kb_desc,
        #     # "Links to reference": kb_urls,
        #     "Service now reference links": kb_content2,
        #     # "Links to reference for ": kb_urls2
        # }

    def extract_from_opensearch(self, query: str):
        start = time.time()
        docs = self._sync_opensearch_search(query)
        print(f"Time for OpenSearch: {time.time() - start:.2f} seconds")
        return [{'Ticket': doc.metadata['ticket_number'], 'Description': [doc.metadata['work_notes']]} for doc in docs]

    def _sync_opensearch_search(self, query: str):
        gh_aoss = OpenSearchVectorSearch(
            index_name="service-now-index",
            embedding_function=embeddings,
            opensearch_url=OPENSEARCH_ENDPOINT,
            http_auth=awsauth,
            timeout=300,
            use_ssl=True,
            connection_class=RequestsHttpConnection,
        )
        return gh_aoss.similarity_search(
            query=query,
            k=2,
            search_type="script_scoring",
            vector_field="embedding",
            text_field="work_notes",
            metadata_field="*",
            score_threshold=0.6
        )

    # def extract_from_kb(self, query: str):
    #     start = time.time()
    #     parsed_data = json.loads(kb_tool.run(query))
    #     content_list = [result["content"] for result in parsed_data["results"] if result['score'] > 0.60] 
    #     desc = [str(result["metadata"].get("x-amz-bedrock-kb-description", "")) for result in parsed_data["results"] if result['score'] > 0.60]
    #     url_list = [result["source_uri"] for result in parsed_data["results"] if result['score'] > 0.60]
    #     answer = "\n".join(content_list), "\n".join(desc), url_list
    #     print(f"Time for Bedrock KB: {time.time() - start:.2f} seconds")
    #     return answer
    
    def extract_from_kb(self, query: str):
        start = time.time()
        parsed_data = json.loads(kb_tool.run(query))
        results = [
            {
                "content": result["content"],
                "description": str(result["metadata"].get("x-amz-bedrock-kb-description", "")),
                "url": result["source_uri"]
            }
            for result in parsed_data["results"]
        ]
        print(f"Time for Bedrock KB: {time.time() - start:.2f} seconds")
        return results

    # def extract_from_kb2(self, query: str):
    #     start = time.time()
    #     query1="get cbre.service-now.com links for" + query
    #     parsed_data = json.loads(kb_tool2.run(query1))
    #     content_list = [result["content"] for result in parsed_data["results"] ]
    #     url_list = [result["source_uri"] for result in parsed_data["results"]]
    #     answer = "\n".join(content_list), url_list
    #     print(f"Time for KB2: {time.time() - start:.2f} seconds")
    #     return answer
    def extract_from_kb2(self, query: str):
        start = time.time()
        query1 = "get cbre.service-now.com links for " + query + " from the Cloud Assistant Bot SNOW Form Links"
        parsed_data = json.loads(kb_tool2.run(query1))
        results = [
            {
                "content": result["content"],
                "url": result["source_uri"]
            }
            for result in parsed_data["results"]
        ]
        print(f"Time for KB2: {time.time() - start:.2f} seconds")
        return results

# A = RetrieveContentToolInput(data="service now ticket aws")
# gh_docs = RetrieveContentTool()._run(A.data)
# print(gh_docs)

/Users/SKotra/Downloads/test_files/.venv/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(
/Users/SKotra/Downloads/test_files/.venv/lib/python3.13/site-packages/requests_aws4auth/aws4signingkey.py:85: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  self.date = date or datetime.utcnow().strftime('%Y%m%d')


In [4]:
tool = RetrieveContentTool()
# tool._run(data='Cloud FOrm')

In [5]:
# Apply the function to each row
import html

for index, row in dataframe.iterrows():
    question = row["user_input"]
    answers, times = get_responses_with_time(question)
    tool_result = tool._run(data=question)
    cleaned_output = html.unescape(tool_result)
    # print(answers, times, tool_result)
    print(tool_result)
    dataframe.at[index, 'cleaned_output'] = str(cleaned_output)
    dataframe.at[index, "ai_response"] = answers.get("http://127.0.0.1:8000/ask", "")
    dataframe.at[index, "ai_time"] = times.get("http://127.0.0.1:8000/ask", 0.0)

Using Tool: Bedrock Knowledge Base Retriever Tool
Using Tool: Bedrock Knowledge Base Retriever Tool


/Users/SKotra/Downloads/test_files/.venv/lib/python3.13/site-packages/botocore/auth.py:422: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  datetime_now = datetime.datetime.utcnow()
/Users/SKotra/Downloads/test_files/.venv/lib/python3.13/site-packages/requests_aws4auth/aws4auth.py:384: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.datetime.utcnow()


Time for KB2: 0.58 seconds
Time for OpenSearch: 0.64 seconds
Time for Bedrock KB: 0.86 seconds
Total time taken for all operations: 0.87 seconds
=== Confluence & SharePoint Results ===
Content: # Overview If you require a service to allow sending email from your application, Cloud Engineering offers SendGrid as a Service. The platform is hosted in our Azure 2.0 shared Shared NonProd and Shared Prod subscriptions. # Availability * All cloud providers * All CBRE approved regions globally # Features * CBRE Enterprise Grade email delivery via cbre.com * SPF authentication * DKIM signing * DMARC validation
Description: 
Link: https://cbre-jira.atlassian.net/wiki/spaces/CLOUD/pages/39584092/Twilio+SendGrid+Enterprise+Mail
---
Content: 
Description: 58
# CHAPTER 2 *Cloud-native application challenges*
## *2.4.4 Dealing with inconsistent data*
Having stored data in a relational data store like PostgreSQL or a NoSQL approach like Redis doesn't solve the problem of having inconsistent data acros

In [6]:
dataframe

,user_input,ground_truth,cleaned_output,ai_response,ai_time
0,I would like to set up email services for my a...,"Yes, Cloud Engineering offers a shared SendGri...",=== Confluence & SharePoint Results ===\nConte...,**To set up email services for your applicatio...,6.293
1,"I need access to this TFE project, can you hav...",Please raise ServiceNow ticket to get new TFE ...,=== Confluence & SharePoint Results ===\nConte...,"**To request access to a TFE project, please f...",6.196
2,What are the supported AWS services?,"EC2, SQL Server (IaaS), RDS, EFS/FSX , S3",=== Confluence & SharePoint Results ===\nConte...,**Supported AWS Services:**\n\nThe following A...,7.336
3,Are there any Azure services that will be supp...,Azure PostgreSQL (Will be available by end of ...,=== Confluence & SharePoint Results ===\nConte...,"**Yes, there are Azure services that will be s...",5.552
4,What is the BaaS architecture for a Dedicated ...,Transaction log backups run every 15 minutes f...,=== Confluence & SharePoint Results ===\nConte...,**BaaS Architecture for a Dedicated Subscripti...,7.114
5,Is there a RACI for this service?,"For BaaS service, the RACI matrix shows clear ...",=== Confluence & SharePoint Results ===\nConte...,**No relevant RACI information was found for t...,4.468
6,What is the SLA for a restoration request?,The SLA for restoration requests varies by app...,=== Confluence & SharePoint Results ===\nConte...,**SLA for a Restoration Request:**\n\n- **Serv...,4.553
7,What are the latest changes that happened to t...,"The last tagging policy was updated on May 15,...",=== Confluence & SharePoint Results ===\nConte...,The latest changes to the tagging are as follo...,7.323
8,What is CloudHealth and tell me more about its...,Cloud Health provides a single solution for mu...,=== Confluence & SharePoint Results ===\nConte...,**CloudHealth Overview**\n\nCloudHealth provid...,6.619


In [7]:
# Save the updated DataFrame to a new Excel file
dataframe.to_csv("nova_pro_optimized_generated_data.csv", index=False)
print("Responses and timings saved to generated_data.xlsx")

Responses and timings saved to generated_data.xlsx


In [69]:
import boto3
import pandas as pd
from crewai import LLM

# Initialize Bedrock runtime and LLM
bedrock_runtime = boto3.client("bedrock-runtime", region_name="us-east-1")
llm = LLM(model="bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0")

# Load Excel file
dataframe = pd.read_excel("generated_data.xlsx")

# Output file
filename = "evaluation_results.txt"
with open(filename, "w") as f:
    f.write("LLM Evaluation Results for Dev, QA, and Prod\n\n")

# Prompt template
def create_judge_prompt(question, ground_answer, dev_answer, qa_answer, prod_answer, dev_time, qa_time, prod_time, context):
    return f"""
You are an expert evaluator tasked with assessing the quality of AI-generated answers based on a given question, the ground truth answer, and the context retrieved from a tool or database.

Below are the inputs:

Question:
{question}

Ground Truth Answer:
{ground_answer}

Tool/Context provided to the LLM:
{context}

AI-Generated Answers:
- Dev Environment Answer: {dev_answer}
  - Dev time: {dev_time}
- QA Environment Answer: {qa_answer}
  - QA time: {qa_time}
- Prod Environment Answer: {prod_answer}
  - Prod time: {prod_time}

Your task is to evaluate each of the three AI-generated answers (Dev, QA, and Prod) in comparison to the ground truth and context. For each answer, assess:
- Relevance to the question
- Completeness compared to the ground truth
- Accuracy as per the context

Return your evaluation in the following JSON format:

{{
  "Dev": {{
    "is_correct": true/false,
    "reasoning": "Your reasoning here",
    "metrics": {{
      "relevance": 0-1,
      "completeness": 0-1,
      "accuracy": 0-1
    }}
  }},
  "QA": {{
    "is_correct": true/false,
    "reasoning": "Your reasoning here",
    "metrics": {{
      "relevance": 0-1,
      "completeness": 0-1,
      "accuracy": 0-1
    }}
  }},
  "Prod": {{
    "is_correct": true/false,
    "reasoning": "Your reasoning here",
    "metrics": {{
      "relevance": 0-1,
      "completeness": 0-1,
      "accuracy": 0-1
    }}
  }}

  Overall Observation on which response seems the best and why. Reason properly. Make a comparison between the 3 for each question, which works better.
}}
"""

# Evaluate each row
for i, row in dataframe.iterrows():
    question = row["user_input"]
    ground = row["ground_truth"]
    context = row["tool_response"]

    dev_answer = row["dev_response"]
    qa_answer = row["qa_response"]
    prod_answer = row["prod_response"]

    dev_time = row["dev_time"]
    qa_time = row["qa_time"]
    prod_time = row["prod_time"]

    prompt = create_judge_prompt(question, ground, dev_answer, qa_answer, prod_answer, dev_time, qa_time, prod_time, context)
    print(f"Evaluating Question {i}")
    evaluation = llm.call(prompt)

    # Write to file
    with open(filename, "a") as file:
        file.write(f"--- Question {i} ---\n")
        file.write(f"Dev Time: {dev_time}s | QA Time: {qa_time}s | Prod Time: {prod_time}s\n")
        file.write(f"Evaluation:\n{evaluation}\n")
        file.write("\n" + "-"*60 + "\n\n")

print(f"All evaluations written to {filename}")


KeyError: 'tool_response'